# Execution Flow of MLflow Compatible Agent: A Step-by-Step Deep Dive

## Introduction

Understanding how an MLflow-compatible agent executes internally is crucial for debugging, optimization, and building production-ready AI systems. In this article, we'll trace through every single step of execution, from the moment a user makes a request to when they receive a response.

We'll use a real example with actual code from the GitHub repository and follow the execution flow line by line, showing exactly what happens at each stage.

## Sample Request Setup

Let's start with our example request:

```python
# User asks a question that requires tool usage
user_query = "What is the latest news on OpenAI product releases? Provide results in bullet points"
```

This query will trigger:
1. An LLM call to understand the query
2. A tool call to search the web
3. Another LLM call to format the results

Let's trace through the complete execution!

## Complete Agent Code

First, let's review the complete MLflow-compatible agent code:

```python
import mlflow
from typing import Annotated, Optional, Any
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from databricks_langchain import ChatDatabricks
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langgraph.prebuilt.tool_node import tools_condition
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import ChatAgentMessage, ChatAgentResponse, ChatContext
from mlflow.models import ModelConfig

# Step 1: Create the agent graph
def create_tool_calling_agent(model, tools):
    """
    Creates a LangGraph-based tool-calling agent with MLflow tracing
    """
    llm_with_tools = model.bind_tools(tools=tools)
    
    # Preprocessor to extract messages from state
    preprocessor = RunnableLambda(lambda state: state["messages"])
    model_runnable = preprocessor | llm_with_tools

    def tool_calling_llm(state: ChatAgentState, config: RunnableConfig):
        """Node function that calls the LLM"""
        response = model_runnable.invoke(state, config)
        return {"messages": [response]}
    
    # Build the graph
    builder = StateGraph(ChatAgentState)
    builder.add_node("tool_calling_llm", RunnableLambda(tool_calling_llm))
    builder.add_node("tools", ChatAgentToolNode(tools=tools))
    builder.add_edge(START, "tool_calling_llm")
    
    builder.add_conditional_edges(
        "tool_calling_llm",
        tools_condition,
        ["tools", END]
    )
    builder.add_edge("tools", "tool_calling_llm")
    
    return builder.compile()

# Step 2: Create the MLflow-compatible agent class
class DocsAgent(ChatAgent):
    def __init__(self, config, tools):
        self.config = ModelConfig(development_config=config)
        self.tools = tools
        self.agent = self._build_agent_from_config()

    def _build_agent_from_config(self):
        llm = ChatDatabricks(
            endpoint=self.config.get("endpoint_name"),
            temperature=self.config.get("temperature"),
            max_tokens=self.config.get("max_tokens"),
        )
        agent = create_tool_calling_agent(llm, tools=self.tools)
        return agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        request = {"messages": self._convert_messages_to_dict(messages)}
        output = self.agent.invoke(request)
        return ChatAgentResponse(**output)

# Step 3: Initialize the agent
uc_client = DatabricksFunctionClient()
set_uc_function_client(uc_client)

catalog = "agentic_ai"
schema = "databricks"
LLM_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"

baseline_config = {
    "endpoint_name": LLM_ENDPOINT,
    "temperature": 0.01,
    "max_tokens": 1000
}

uc_tool_names = [f"{catalog}.{schema}.search_web"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools = [*uc_toolkit.tools]

AGENT = DocsAgent(baseline_config, tools)
```

## Execution Flow: Step-by-Step Breakdown

Now let's execute our sample request and trace through every step:

```python
# Enable MLflow tracing
mlflow.langchain.autolog()
mlflow.set_experiment("Agent_Execution_Flow_Demo")

# Execute the request
result = AGENT.predict([{
    "role": "user",
    "content": "What is the latest news on OpenAI product releases? Provide results in bullet points"
}])
```

### Phase 1: Request Initiation (AGENT.predict)

**Code Entry Point:**
```python
def predict(
    self,
    messages: list[ChatAgentMessage],
    context: Optional[ChatContext] = None,
    custom_inputs: Optional[dict[str, Any]] = None,
) -> ChatAgentResponse:
```

**What Happens:**

**Step 1.1: Method Invocation**
```python
# User calls:
result = AGENT.predict([{
    "role": "user",
    "content": "What is the latest news on OpenAI product releases? Provide results in bullet points"
}])

# Internally, the messages parameter receives:
messages = [
    {
        "role": "user",
        "content": "What is the latest news on OpenAI product releases? Provide results in bullet points"
    }
]
```

**MLflow Action:** Creates Root Trace
```python
# MLflow automatically creates a trace
trace = {
    "trace_id": "tr_xyz123abc",
    "request_time": 1732400000000,
    "state": "IN_PROGRESS"
}

# Creates root span for the predict method
root_span = {
    "span_id": "sp_001",
    "name": "DocsAgent.predict",
    "span_type": "AGENT",
    "start_time": 1732400000000,
    "status": "running"
}
```

**Step 1.2: Message Conversion**
```python
request = {"messages": self._convert_messages_to_dict(messages)}

# After conversion:
request = {
    "messages": [
        {
            "role": "user",
            "content": "What is the latest news on OpenAI product releases? Provide results in bullet points",
            "type": "human"
        }
    ]
}
```

**Step 1.3: Agent Invocation**
```python
output = self.agent.invoke(request)
# This triggers the LangGraph execution
```

### Phase 2: LangGraph Initialization

**Code Entry Point:**
```python
# self.agent is a CompiledGraph created by:
agent = create_tool_calling_agent(llm, tools=self.tools)
```

**What Happens:**

**Step 2.1: Graph Receives Request**
```python
# The compiled graph's invoke method is called
compiled_graph.invoke(request)

# Initial state is created:
initial_state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?")
    ]
}
```

**MLflow Action:** Creates Chain Span
```python
# MLflow creates a span for the graph execution
graph_span = {
    "span_id": "sp_002",
    "parent_span_id": "sp_001",
    "name": "StateGraph.invoke",
    "span_type": "CHAIN",
    "start_time": 1732400000050,
    "inputs": {
        "messages": [...]
    }
}
```

**Step 2.2: Graph Determines Starting Node**
```python
# Graph starts at START edge
# START edge points to "tool_calling_llm" node
current_node = "tool_calling_llm"
```

### Phase 3: First LLM Call (Decision Making)

**Code Entry Point:**
```python
def tool_calling_llm(state: ChatAgentState, config: RunnableConfig):
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

**What Happens:**

**Step 3.1: Node Function Called**
```python
# LangGraph calls the tool_calling_llm function
# Parameters:
state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?")
    ]
}

config = RunnableConfig(
    configurable={
        "temperature": 0.01,
        "max_tokens": 1000
    },
    metadata={
        "trace_id": "tr_xyz123abc",
        "span_id": "sp_003"
    }
)
```

**Step 3.2: Preprocessor Extraction**
```python
# The preprocessor extracts messages from state
preprocessor = RunnableLambda(lambda state: state["messages"])

# Execution:
messages = preprocessor.invoke(state, config)
# Returns: [HumanMessage(content="What is the latest news...")]
```

**Step 3.3: Model Invocation**
```python
# model_runnable = preprocessor | llm_with_tools
# The pipe (|) chains the operations

# First: preprocessor extracts messages
extracted_messages = [HumanMessage(content="What is the latest news...")]

# Second: llm_with_tools processes the messages
response = llm_with_tools.invoke(extracted_messages, config)
```

**MLflow Action:** Creates LLM Span
```python
# MLflow creates a span for the LLM call
llm_span = {
    "span_id": "sp_003",
    "parent_span_id": "sp_002",
    "name": "tool_calling_llm",
    "span_type": "LLM",
    "start_time": 1732400000100,
    "inputs": {
        "messages": [
            {"role": "user", "content": "What is the latest news on OpenAI product releases?"}
        ],
        "model": "databricks-meta-llama-3-3-70b-instruct",
        "temperature": 0.01,
        "max_tokens": 1000
    }
}
```

**Step 3.4: LLM Processing**
```python
# The LLM analyzes the request
# Internal LLM reasoning (conceptual):
# 1. User wants latest news about OpenAI
# 2. This requires real-time information
# 3. I have access to search_web tool
# 4. Decision: Call search_web tool

# LLM returns AIMessage with tool_calls
response = AIMessage(
    content="",  # Empty content when calling tools
    tool_calls=[
        {
            "name": "agentic_ai__databricks__search_web",
            "args": {
                "query": "OpenAI latest product releases news 2024"
            },
            "id": "call_abc123"
        }
    ]
)
```

**MLflow Action:** Updates LLM Span
```python
# MLflow updates the span with outputs and metrics
llm_span.update({
    "end_time": 1732400002750,
    "outputs": {
        "content": "",
        "tool_calls": [
            {
                "name": "agentic_ai__databricks__search_web",
                "args": {"query": "OpenAI latest product releases news 2024"},
                "id": "call_abc123"
            }
        ]
    },
    "attributes": {
        "mlflow.chat.tokenUsage": {
            "input_tokens": 67,
            "output_tokens": 45,
            "total_tokens": 112
        },
        "execution_time_ms": 2650,
        "has_tool_calls": true,
        "tool_call_count": 1
    }
})
```

**Step 3.5: Update State**
```python
# The node function returns updated messages
return {"messages": [response]}

# State is updated:
state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?"),
        AIMessage(
            content="",
            tool_calls=[{
                "name": "agentic_ai__databricks__search_web",
                "args": {"query": "OpenAI latest product releases news 2024"},
                "id": "call_abc123"
            }]
        )
    ]
}
```

### Phase 4: Conditional Routing (Decision Point)

**Code Entry Point:**
```python
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,  # This function decides the next step
    ["tools", END]
)
```

**What Happens:**

**Step 4.1: tools_condition Evaluation**
```python
# LangGraph calls tools_condition to decide routing
def tools_condition(state):
    messages = state["messages"]
    last_message = messages[-1]
    
    # Check if last message has tool_calls
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    else:
        return END

# Execution:
last_message = state["messages"][-1]
# last_message is AIMessage with tool_calls

# Decision:
next_node = "tools"  # Route to tools node
```

**MLflow Action:** Logs Routing Decision
```python
# MLflow can log the routing decision as span attributes
llm_span.set_attributes({
    "routing_decision": "tools",
    "reason": "tool_calls_present"
})
```

**Step 4.2: Graph Routes to Tools Node**
```python
# LangGraph routes execution to the "tools" node
current_node = "tools"
```

### Phase 5: Tool Execution

**Code Entry Point:**
```python
builder.add_node("tools", ChatAgentToolNode(tools=tools))
```

**What Happens:**

**Step 5.1: ChatAgentToolNode Initialization**
```python
# ChatAgentToolNode receives the state
tool_node = ChatAgentToolNode(tools=tools)

# State contains:
state = {
    "messages": [
        HumanMessage(content="What is the latest news..."),
        AIMessage(content="", tool_calls=[...])
    ]
}
```

**Step 5.2: Extract Tool Calls**
```python
# ChatAgentToolNode extracts tool calls from last message
messages = state["messages"]
last_message = messages[-1]
tool_calls = last_message.tool_calls

# Extracted tool call:
tool_call = {
    "name": "agentic_ai__databricks__search_web",
    "args": {"query": "OpenAI latest product releases news 2024"},
    "id": "call_abc123"
}
```

**MLflow Action:** Creates Tool Span
```python
# MLflow creates a span for tool execution
tool_span = {
    "span_id": "sp_004",
    "parent_span_id": "sp_002",
    "name": "search_web",
    "span_type": "TOOL",
    "start_time": 1732400002800,
    "inputs": {
        "tool_name": "agentic_ai__databricks__search_web",
        "tool_input": {
            "query": "OpenAI latest product releases news 2024"
        }
    }
}
```

**Step 5.3: Find and Execute Tool**
```python
# Find the matching tool
tool_to_execute = None
for tool in tools:
    if tool.name == "agentic_ai__databricks__search_web":
        tool_to_execute = tool
        break

# Execute the tool
tool_input = {"query": "OpenAI latest product releases news 2024"}
tool_result = tool_to_execute.invoke(tool_input)

# Tool execution (conceptual):
# 1. Connect to search API
# 2. Query: "OpenAI latest product releases news 2024"
# 3. Retrieve results
# 4. Format and return

# Result:
tool_result = """
Recent OpenAI developments include:
- GPT-4 Turbo launched with 128K context window for processing longer documents
- ChatGPT Enterprise released with advanced security features and admin controls
- DALL-E 3 integrated directly into ChatGPT for seamless image generation
- Custom GPTs marketplace announced, allowing users to create specialized AI assistants
- API improvements with better rate limits and new function calling capabilities
"""
```

**MLflow Action:** Updates Tool Span
```python
# MLflow updates the span with results
tool_span.update({
    "end_time": 1732400006150,
    "outputs": {
        "content": "Recent OpenAI developments include:\n- GPT-4 Turbo launched with..."
    },
    "attributes": {
        "mlflow.tool.function_name": "agentic_ai.databricks.search_web",
        "execution_time_ms": 3350,
        "tool_status": "success",
        "result_length": 487
    }
})
```

**Step 5.4: Create Tool Response Message**
```python
# ChatAgentToolNode creates a ToolMessage
tool_message = ToolMessage(
    content=tool_result,
    tool_call_id="call_abc123"
)

# Return updated state
return {"messages": [tool_message]}
```

**Step 5.5: Update State**
```python
# State after tool execution:
state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?"),
        AIMessage(content="", tool_calls=[...]),
        ToolMessage(
            content="Recent OpenAI developments include:\n- GPT-4 Turbo...",
            tool_call_id="call_abc123"
        )
    ]
}
```

### Phase 6: Return to LLM (Response Generation)

**Code Entry Point:**
```python
builder.add_edge("tools", "tool_calling_llm")
# After tools node, always go back to tool_calling_llm
```

**What Happens:**

**Step 6.1: Second LLM Call Initiated**
```python
# LangGraph routes back to tool_calling_llm node
# The same function is called again with updated state

def tool_calling_llm(state: ChatAgentState, config: RunnableConfig):
    response = model_runnable.invoke(state, config)
    return {"messages": [response]}
```

**Step 6.2: State Contains Tool Results**
```python
# State now has three messages:
state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?"),
        AIMessage(content="", tool_calls=[...]),
        ToolMessage(content="Recent OpenAI developments include...")
    ]
}
```

**MLflow Action:** Creates Second LLM Span
```python
# MLflow creates another LLM span
llm_span_2 = {
    "span_id": "sp_005",
    "parent_span_id": "sp_002",
    "name": "tool_calling_llm",
    "span_type": "LLM",
    "start_time": 1732400006200,
    "inputs": {
        "messages": [
            {"role": "user", "content": "What is the latest news..."},
            {"role": "assistant", "content": "", "tool_calls": [...]},
            {"role": "tool", "content": "Recent OpenAI developments...", "tool_call_id": "call_abc123"}
        ],
        "model": "databricks-meta-llama-3-3-70b-instruct",
        "temperature": 0.01,
        "max_tokens": 1000
    }
}
```

**Step 6.3: LLM Processes Tool Results**
```python
# The LLM now has:
# 1. Original user question
# 2. Its decision to call search_web
# 3. The search results

# LLM generates final response
response = AIMessage(
    content="""Based on the latest information, here are the recent OpenAI product releases:

• GPT-4 Turbo with 128K context window for processing longer documents and complex tasks
• ChatGPT Enterprise featuring advanced security, admin controls, and priority access
• DALL-E 3 integration enabling users to generate images directly within ChatGPT conversations
• Custom GPTs marketplace where users can create and share specialized AI assistants
• Enhanced API capabilities with improved rate limits and sophisticated function calling features""",
    tool_calls=[]  # No more tool calls needed
)
```

**MLflow Action:** Updates Second LLM Span
```python
# MLflow updates the span
llm_span_2.update({
    "end_time": 1732400008300,
    "outputs": {
        "content": "Based on the latest information, here are the recent OpenAI product releases:\n\n• GPT-4 Turbo...",
        "tool_calls": []
    },
    "attributes": {
        "mlflow.chat.tokenUsage": {
            "input_tokens": 245,
            "output_tokens": 180,
            "total_tokens": 425
        },
        "execution_time_ms": 2100,
        "has_tool_calls": false,
        "response_type": "text"
    }
})
```

**Step 6.4: Update State with Final Response**
```python
return {"messages": [response]}

# Final state:
state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?"),
        AIMessage(content="", tool_calls=[...]),
        ToolMessage(content="Recent OpenAI developments..."),
        AIMessage(content="Based on the latest information, here are the recent OpenAI product releases:\n\n• GPT-4 Turbo...")
    ]
}
```

### Phase 7: Final Routing Decision

**Code Entry Point:**
```python
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,
    ["tools", END]
)
```

**What Happens:**

**Step 7.1: Evaluate tools_condition Again**
```python
def tools_condition(state):
    messages = state["messages"]
    last_message = messages[-1]
    
    # Check for tool_calls
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    else:
        return END

# Execution:
last_message = AIMessage(content="Based on the latest...", tool_calls=[])
# No tool_calls present

# Decision:
next_node = END  # Terminate execution
```

**Step 7.2: Graph Execution Completes**
```python
# LangGraph marks execution as complete
execution_status = "COMPLETED"
```

**MLflow Action:** Closes Chain Span
```python
# MLflow closes the StateGraph span
graph_span.update({
    "end_time": 1732400008350,
    "outputs": {
        "messages": [/* all four messages */]
    },
    "attributes": {
        "total_nodes_executed": 3,
        "total_edges_traversed": 4,
        "execution_path": "START -> tool_calling_llm -> tools -> tool_calling_llm -> END"
    }
})
```

### Phase 8: Response Assembly

**Code Entry Point:**
```python
def predict(...) -> ChatAgentResponse:
    request = {"messages": self._convert_messages_to_dict(messages)}
    output = self.agent.invoke(request)
    return ChatAgentResponse(**output)  # ← We're here
```

**What Happens:**

**Step 8.1: Extract Output**
```python
# output from agent.invoke():
output = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?"),
        AIMessage(content="", tool_calls=[...]),
        ToolMessage(content="Recent OpenAI developments..."),
        AIMessage(content="Based on the latest information, here are the recent OpenAI product releases:\n\n• GPT-4 Turbo...")
    ]
}
```

**Step 8.2: Create ChatAgentResponse**
```python
# Convert to MLflow's response format
response = ChatAgentResponse(**output)

# ChatAgentResponse structure:
response = {
    "messages": [
        {"role": "user", "content": "What is the latest news on OpenAI product releases?"},
        {"role": "assistant", "content": "", "tool_calls": [...]},
        {"role": "tool", "content": "Recent OpenAI developments...", "tool_call_id": "call_abc123"},
        {"role": "assistant", "content": "Based on the latest information, here are the recent OpenAI product releases:\n\n• GPT-4 Turbo..."}
    ]
}
```

**MLflow Action:** Closes Root Span
```python
# MLflow closes the root DocsAgent.predict span
root_span.update({
    "end_time": 1732400008400,
    "outputs": {
        "messages": [/* all messages */]
    },
    "attributes": {
        "total_execution_time_ms": 8400,
        "message_count": 4,
        "final_message_length": 456
    }
})
```

**Step 8.3: Finalize Trace**
```python
# MLflow finalizes the trace
trace.update({
    "execution_duration": 8400,
    "state": "OK",
    "response_preview": "Based on the latest information, here are the recent OpenAI product releases:\n\n• GPT-4 Turbo...",
    "token_usage": {
        "input_tokens": 312,
        "output_tokens": 225,
        "total_tokens": 537
    }
})
```

**Step 8.4: Return to User**
```python
# The predict method returns
return response

# User receives:
result = ChatAgentResponse(
    messages=[
        {...},  # All messages
        {"role": "assistant", "content": "Based on the latest information, here are the recent OpenAI product releases:\n\n• GPT-4 Turbo..."}
    ]
)
```

## Complete Execution Timeline

Here's a visual representation of the entire execution flow:

```
⏱️ Timeline: Total Duration = 8.4 seconds

T=0.000s   ┌─────────────────────────────────────────────────────┐
           │ Phase 1: AGENT.predict() called                    │
           │ - Create root trace                                 │
           │ - Convert messages                                  │
           └─────────────────────────────────────────────────────┘

T=0.050s   ┌─────────────────────────────────────────────────────┐
           │ Phase 2: LangGraph initialized                      │
           │ - Create initial state                              │
           │ - Route to START -> tool_calling_llm                │
           └─────────────────────────────────────────────────────┘

T=0.100s   ┌─────────────────────────────────────────────────────┐
           │ Phase 3: First LLM Call (2.65s)                     │
           │ - Extract messages with preprocessor                │
T=0.150s   │ - Send to LLM                                       │
           │ - LLM analyzes query                                │
           │ - LLM decides to call search_web                    │
T=2.750s   │ - Return AIMessage with tool_calls                  │
           │ - Token usage: 112 (67 input + 45 output)          │
           └─────────────────────────────────────────────────────┘

T=2.800s   ┌─────────────────────────────────────────────────────┐
           │ Phase 4: Conditional Routing                        │
           │ - tools_condition evaluates state                   │
           │ - Detects tool_calls present                        │
           │ - Routes to "tools" node                            │
           └─────────────────────────────────────────────────────┘

T=2.850s   ┌─────────────────────────────────────────────────────┐
           │ Phase 5: Tool Execution (3.35s)                     │
           │ - Extract tool call from AIMessage                  │
           │ - Find search_web tool                              │
T=3.000s   │ - Execute: search_web("OpenAI latest...")           │
           │ - Wait for search results                           │
T=6.150s   │ - Return ToolMessage with results                   │
           │ - Update state with tool response                   │
           └─────────────────────────────────────────────────────┘

T=6.200s   ┌─────────────────────────────────────────────────────┐
           │ Phase 6: Second LLM Call (2.10s)                    │
           │ - Route back to tool_calling_llm                    │
           │ - State now contains tool results                   │
T=6.250s   │ - Send complete conversation to LLM                 │
           │ - LLM processes tool results                        │
           │ - LLM generates formatted response                  │
T=8.300s   │ - Return AIMessage with final answer                │
           │ - Token usage: 425 (245 input + 180 output)        │
           └─────────────────────────────────────────────────────┘

T=8.350s   ┌─────────────────────────────────────────────────────┐
           │ Phase 7: Final Routing                              │
           │ - tools_condition evaluates state                   │
           │ - No tool_calls detected                            │
           │ - Routes to END                                     │
           └─────────────────────────────────────────────────────┘

T=8.400s   ┌─────────────────────────────────────────────────────┐
           │ Phase 8: Response Assembly                          │
           │ - Create ChatAgentResponse                          │
           │ - Finalize trace and spans                          │
           │ - Return to user                                    │
           └─────────────────────────────────────────────────────┘

📊 Final Statistics:
   - Total Duration: 8.4 seconds
   - Total Tokens: 537
   - LLM Calls: 2
   - Tool Calls: 1
   - Spans Created: 5
```

## State Evolution Throughout Execution

Let's see how the state changes at each phase:

```python
# Phase 1: Initial State
state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?")
    ]
}

# Phase 3: After First LLM Call
state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?"),
        AIMessage(
            content="",
            tool_calls=[{
                "name": "agentic_ai__databricks__search_web",
                "args": {"query": "OpenAI latest product releases news 2024"},
                "id": "call_abc123"
            }]
        )
    ]
}

# Phase 5: After Tool Execution
state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?"),
        AIMessage(content="", tool_calls=[...]),
        ToolMessage(
            content="Recent OpenAI developments include:\n- GPT-4 Turbo launched...",
            tool_call_id="call_abc123"
        )
    ]
}

# Phase 6: After Second LLM Call (Final State)
state = {
    "messages": [
        HumanMessage(content="What is the latest news on OpenAI product releases?"),
        AIMessage(content="", tool_calls=[...]),
        ToolMessage(content="Recent OpenAI developments..."),
        AIMessage(
            content="Based on the latest information, here are the recent OpenAI product releases:\n\n• GPT-4 Turbo with 128K context window..."
        )
    ]
}
```

## Key Observations

### 1. **Automatic Tracing**
MLflow automatically created 5 spans without any manual instrumentation:
- 1 Root AGENT span (DocsAgent.predict)
- 1 CHAIN span (StateGraph.invoke)
- 2 LLM spans (two tool_calling_llm calls)
- 1 TOOL span (search_web execution)

### 2. **Configuration Flow**
The `RunnableConfig` flowed through every step:
```python
# Started in predict() → 
# Passed to agent.invoke() → 
# Propagated to tool_calling_llm() → 
# Flowed through model_runnable → 
# Used by LLM for temperature, max_tokens
```

### 3. **State Management**
The `ChatAgentState` accumulated messages:
- Started with 1 message (user query)
- Added AIMessage with tool_calls
- Added ToolMessage with results
- Added final AIMessage with answer
- Each node could access complete conversation history

### 4. **Conditional Routing**
The `tools_condition` function made two decisions:
- First: Detected tool_calls → routed to "tools"
- Second: No tool_calls → routed to END

### 5. **Token Tracking**
MLflow automatically tracked tokens:
- First LLM call: 112 tokens (67 input + 45 output)
- Second LLM call: 425 tokens (245 input + 180 output)
- Total: 537 tokens

## Accessing Execution Details

After execution, you can analyze the complete flow:

```python
# Get the trace
trace_id = mlflow.get_last_active_trace_id()
trace = mlflow.get_trace(trace_id)

# Print execution summary
print(f"Total Duration: {trace.info.execution_duration}ms")
print(f"Total Tokens: {trace.info.token_usage['total_tokens']}")
print(f"Status: {trace.info.state}")

# Analyze each span
print("\n=== Execution Breakdown ===")
for span in trace.data.spans:
    duration = span.end_time_unix_ms - span.start_time_unix_ms
    print(f"\n{span.name} ({span.span_type})")
    print(f"  Duration: {duration}ms")
    
    if span.span_type == "LLM":
        tokens = span.get_attribute("mlflow.chat.tokenUsage")
        print(f"  Tokens: {tokens['total_tokens']}")
        
    if span.span_type == "TOOL":
        tool_name = span.get_attribute("mlflow.tool.function_name")
        print(f"  Tool: {tool_name}")

# Output:
# Total Duration: 8400ms
# Total Tokens: 537
# Status: OK
#
# === Execution Breakdown ===
#
# DocsAgent.predict (AGENT)
#   Duration: 8400ms
#
# StateGraph.invoke (CHAIN)
#   Duration: 8300ms
#
# tool_calling_llm (LLM)
#   Duration: 2650ms
#   Tokens: 112
#
# search_web (TOOL)
#   Duration: 3350ms
#   Tool: agentic_ai.databricks.search_web
#
# tool_calling_llm (LLM)
#   Duration: 2100ms
#   Tokens: 425
```

## Conclusion

The MLflow-compatible agent execution flow demonstrates:

1. **Seamless Integration**: MLflow tracing works automatically without manual instrumentation
2. **Complete Observability**: Every step is captured in detailed spans
3. **Configuration Propagation**: `RunnableConfig` flows through the entire execution chain
4. **State Management**: `ChatAgentState` maintains conversation context throughout
5. **Conditional Logic**: The graph dynamically routes based on LLM decisions
6. **Token Tracking**: Automatic cost and usage monitoring

Understanding this execution flow empowers you to:
- Debug issues by identifying exact failure points
- Optimize performance by finding bottlenecks
- Track costs by monitoring token usage
- Improve agent behavior by analyzing decision patterns

This deep understanding transforms agent development from guesswork to precision engineering, enabling you to build truly production-ready AI systems.

---

**Key Takeaway**: The magic of MLflow-compatible agents isn't just in automatic tracing—it's in the complete, transparent view of every decision, every call, and every transformation your agent makes from question to answer.

Happy building! 🚀